In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [52]:
##########################
save_history = False # if true, updates portfolio and platform history files.
##########################
today = pd.to_datetime('today').normalize()
loans_df = pd.read_csv('data/concatenated.csv')
cash_df = pd.read_csv('data/cash.csv', index_col='platform_account_name')
transactions_df = pd.read_csv('data/transactions.csv')
portfolio_df = pd.read_csv('data/portfolio_history.csv', index_col=0, parse_dates=[0])
platform_dict = { 'loanpad':'Loanpad 60d Access','proplend':'Proplend','crowdproperty':'CrowdProperty AutoInvest','invest_fund':'Invest & Fund','assetz_capital':'Assetz Capital 90d Access','kuflink':'Kuflink 24m Term 28133' }
col_list = ['Paid in capital','Int bearing','Non bearing','Accr int','Residual value','Dist','DPI','RVPI','TVPI','No loans','Avg loan amt','Avg net AER','Blended LTV','Avg term','Max term','Avg ttm','Max ttm','Avg ext','Ext 90d','Non-perform','Long default','Short default','Int default']
platform_totals_list = [loans_df['loan_part_amt'].sum() + cash_df['ib_cash'].sum(),cash_df['nonib_cash'].sum(),loans_df['loan_part_amt'].sum() + cash_df['ib_cash'].sum() + cash_df['nonib_cash'].sum(),loans_df['loan_portion'].sum(),loans_df['loan_part_amt'].sum() / loans_df['loan_portion'].sum(),loans_df['portfolio_weighted_net_aer'].sum(),loans_df['portfolio_weighted_blended_ltv'].sum(),loans_df['portfolio_weighted_term_months'].sum(),loans_df['loan_part_term_months'].max(),loans_df['portfolio_weighted_ttm_months'].sum(),loans_df['loan_part_ttm_days'].max()/33.437,loans_df['portfolio_weighted_maturity_extended_days'].sum(),loans_df.loc[loans_df['loan_part_maturity_extended_days'] > 90, 'loan_part_amt'].sum() / loans_df['loan_part_amt'].sum(),loans_df.loc[loans_df['loan_part_status'].isin(['Short Default', 'Long Default', 'Interest Default']), 'loan_part_amt'].sum() / loans_df['loan_part_amt'].sum(),loans_df.loc[loans_df['loan_part_status'] == 'Long Default', 'loan_part_amt'].sum(),loans_df.loc[loans_df['loan_part_status'] == 'Short Default', 'loan_part_amt'].sum(),loans_df.loc[loans_df['loan_part_status'] == 'Interest Default', 'loan_part_amt'].sum(), loans_df.loc[loans_df['platform_account_sector'] == 'Development']['loan_part_amt'].sum()/loans_df['loan_part_amt'].sum(),loans_df.loc[loans_df['platform_account_sector'] == 'Commercial']['loan_part_amt'].sum()/loans_df['loan_part_amt'].sum(),len(platform_dict),loans_df['loan_part_amt'].sum() / len(platform_dict),
                       1,2,3]
format_dict = { 'Paid in capital': '£{0:,.2f}','Int bearing': '£{0:,.2f}','Non bearing': '£{0:,.2f}','Accr int': '£{0:,.2f}','Residual value': '£{0:,.2f}','Dist': '£{0:,.2f}','DPI': '{0:,.2f}', 'RVPI': '{0:,.2f}', 'TVPI' : '{0:,.2f}','No loans': '{:,.0f}','Avg loan amt': '£{0:,.2f}','Blended LTV': '{:,.0%}','Avg net AER': '{:,.1%}','Avg term': '{:,.0f}','Max term': '{:,.0f}','Avg ttm': '{:,.0f}','Max ttm': '{:,.0f}','Avg ext': '{:,.0f}','Ext 90d': '{:,.0%}','Non-perform': '{:,.0%}','Long default': '£{0:,.0f}','Short default': '£{0:,.0f}','Int default': '£{0:,.0f}','Dev sector': '{:.0%}', 'Comm sector': '{:.0%}','No a/c': '{:,.0f}','Avg a/c amt': '£{0:,.0f}'}
colours_list = ['green','blue','red','orange','purple','pink']
def save_platform_csv(platform): # function to update platform_history.csv for all platform accounts
    exec(platform + "_df = pd.read_csv('data/" + platform + "_history.csv', index_col=0, parse_dates=[0])") # read platform _history.csv
    exec(platform + "_df.loc[today] = platform_row") # append platform row
    exec(platform + "_df.to_csv('data/" + platform + "_history.csv', index=True)") # save platform _history.csv

### Platforms

In [60]:
platform_df = pd.DataFrame(columns=col_list)
for platform in platform_dict:
    platform_account_name = platform_dict[platform]
    platform_loans = loans_df.loc[(loans_df['platform_account_name'] == platform_account_name) & (loans_df['loan_part_status'] != 'Repaid')]
    platform_ib_cash = cash_df.loc[platform_account_name]['ib_cash']
    platform_nonib_cash = cash_df.loc[platform_account_name]['nonib_cash']
    platform_accrued_int = cash_df.loc[platform_account_name]['accrued_int']
    platform_deployed_amt = platform_loans['loan_part_amt'].sum() + platform_ib_cash
    platform_residual_value = platform_deployed_amt + platform_nonib_cash + platform_accrued_int
    platform_extended_pct = platform_loans.loc[platform_loans['loan_part_maturity_extended_days'] > 90, 'loan_part_amt'].sum() / platform_deployed_amt
    platform_nonperform_pct = platform_loans.loc[platform_loans['loan_part_status'].isin(['Short Default', 'Long Default', 'Interest Default']), 'loan_part_amt'].sum() / platform_deployed_amt
    platform_avg_loan_amt = platform_loans['loan_part_amt'].sum() / platform_loans['loan_portion'].sum()
    platform_transactions = transactions_df.loc[transactions_df['platform_account_name'] == platform_account_name]
    platform_paid_in = platform_transactions.loc[platform_transactions['trx_type'] == 'Paid in', 'trx_amt'].abs().sum()
    platform_distributions = platform_transactions.loc[platform_transactions['trx_type'] == 'Distribution', 'trx_amt'].abs().sum()
    platform_dpi = np.where(platform_distributions != 0, platform_distributions/platform_paid_in, 0)
    platform_rvpi = platform_residual_value / platform_paid_in
    platform_row = [platform_paid_in, platform_deployed_amt, platform_nonib_cash, platform_accrued_int, platform_residual_value, platform_distributions, platform_dpi, platform_rvpi, platform_dpi + platform_rvpi, platform_loans['loan_portion'].sum(), platform_avg_loan_amt, platform_loans['platform_weighted_net_aer'].sum(), platform_loans['platform_weighted_blended_ltv'].sum(), platform_loans['platform_weighted_term_months'].sum(), platform_loans['loan_part_term_months'].max(), platform_loans['platform_weighted_ttm_months'].sum(), platform_loans['loan_part_ttm_days'].max()/33.437, platform_loans['platform_weighted_maturity_extended_days'].sum(), platform_extended_pct, platform_nonperform_pct, platform_loans.loc[platform_loans['loan_part_status'] == 'Long Default', 'loan_part_amt'].sum(),platform_loans.loc[platform_loans['loan_part_status'] == 'Short Default', 'loan_part_amt'].sum(),platform_loans.loc[platform_loans['loan_part_status'] == 'Interest Default', 'loan_part_amt'].sum(),
                     ]
    if save_history == True:
        save_platform_csv(platform) # function to update platform_history.csv for all platform accounts
    platform_df.loc[platform_account_name] = platform_row
platform_df.sort_values(by='Residual value', ascending=False).style.format(format_dict)

,Paid in capital,Int bearing,Non bearing,Accr int,Residual value,Dist,DPI,RVPI,TVPI,No loans,Avg loan amt,Avg net AER,Blended LTV,Avg term,Max term,Avg ttm,Max ttm,Avg ext,Ext 90d,Non-perform,Long default,Short default,Int default
Loanpad 60d Access,"£5,000.00","£5,580.00",£7.14,£0.00,"£5,587.14",£0.00,0.00,1.12,1.12,227,£24.26,6.6%,44%,2,2,2,2,90,31%,1%,£0,£0,£41
Invest & Fund,"£2,900.00","£3,117.30",£4.54,£97.64,"£3,219.48",£0.00,0.00,1.11,1.11,20,£155.87,7.2%,66%,11,24,6,15,3,1%,0%,£0,£0,£0
Proplend,"£3,000.00","£3,000.00",£5.45,£0.00,"£3,005.45",£291.61,0.10,1.00,1.10,3,"£1,000.00",6.7%,50%,28,36,16,24,0,0%,67%,£0,"£1,000","£1,000"
CrowdProperty AutoInvest,"£2,000.00","£1,717.85",£0.00,£0.00,"£1,717.85",£389.91,0.19,0.86,1.05,30,£57.26,7.8%,62%,15,23,6,14,0,0%,67%,£979,£169,£0
Kuflink 24m Term 28133,"£1,000.00","£1,050.99",£0.00,£93.39,"£1,144.38",£0.00,0.00,1.14,1.14,145,£7.20,7.9%,61%,24,24,11,10,141,38%,19%,£0,£0,£197
Assetz Capital 90d Access,"£1,009.61",£680.35,£46.10,£0.00,£726.45,£315.78,0.31,0.72,1.03,162,£4.20,4.1%,59%,53,103,16,43,0,0%,19%,£19,£28,£80


### Portfolio

In [ ]:
portfolio_df.loc[today] = platform_totals_list
if save_history == True:
    portfolio_df.to_csv('data/portfolio_history.csv', index=True)
portfolio_df.sort_index(ascending=False).style.format_index(lambda s: s.strftime('%Y-%m')).format(format_dict)

In [ ]:
ax = platform_df.plot.barh(y='Avg loan amt', legend=False)
ax.bar_label(ax.containers[0], fmt='£{0:,.2f}', label_type='edge', padding=5)

In [ ]:
ax = platform_df['Total amt'].plot(kind='pie', colors=colours_list, autopct='%1.0f%%', legend=False)

In [ ]:
ax = platform_df.plot(kind='scatter', x='Blended LTV', y='Avg net AER', s=150, c=colours_list)